# 정확도

$ 정확도(accuracy) = \frac{예측값에서 정답과 같은 수}{정답} $

하지만 __이진 분류__ 와 __불균형 데이터__ 에서는 좋은 지표가 아님

In [1]:
import numpy as np
from sklearn.base import BaseEstimator # BaseEstimator라는 클래스를 가져옴

class MyDummyClassifier(BaseEstimator): # BaseEstimator를 상속 받음
    # fit() 메소드는 아무것도 학습하지 않음.
    def fit(self, X, y=None):
        pass
    
    # predict() 메소드는 단순히 Sex feature가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1)) # X.shape에서 행 값과 1 크기의 0인 값들의 배열을 만듦
        for i in range(X.shape[0]): # X 행 길이 만큼 반복
            if X['Sex'].iloc[i] == 1: # 'Sex 값이 1이면
                pred[i] = 0 # 예측 값을 0으로
            else:
                pred[i] = 1 #아니면 1로 함
                
        return pred
        

In [2]:
# 위에서 진행한 전처리 내용들을 정리하여 함수로 재구성

def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    
    return df

In [3]:
# 머신러닝 알고리즘에 불필요한 속성 제거

def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True) # axis=1은 열 방향으로 drop하라는 뜻, inplace는 실제 데이터에 적용함
    return df

In [4]:
# 레이블 인코딩 수행. object 데이터를 수치형 클래스로 바꿔줌
from sklearn.preprocessing import LabelEncoder

def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder() # 클래스 객체화
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
        
    return df

In [5]:
# 앞에서 설정한 데이터 전처리 함수 호출

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
# 원본 데이터를 재로딩, 데이터 가공, 학습데이터/테스트 데이터 분할
titanic_df = pd.read_csv('../chapter02/titanic_train.csv')

In [8]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
y_titanic_df = titanic_df['Survived'] # 정답 값
X_titanic_df = titanic_df.drop('Survived', axis=1) # 정답 열을 뺀 나머지 데이터
X_titanic_df = transform_features(X_titanic_df) # 위에서 설정한 전처리를 모두 진행함
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)


In [10]:
myclf = MyDummyClassifier() # 
myclf.fit(X_train, y_train) # 실제로는 학습을 진행 안함

mypredictions = myclf.predict(X_test) # 예측 실행, 실제로는 성별만 본다
print('Dummy Classifier의 정확도는: {:.4f}' .format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는: 0.7877


In [11]:
# mnist를 통해 불균형 데이터에서 정확도 예측의 부적합성 확인

from sklearn.datasets import load_digits # mnist데이터셋 
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator # 
from sklearn.metrics import accuracy_score

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    # 입력값으로 들어오는 X 데이터 셋의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)
    
# 사이킷런의 내장 데이터 셋인 load_digits()를 이용하여 MNIST 데이터 로딩
digits = load_digits()

print(digits.data)
print(digits.data.shape)
print(digits.target)
print(digits.target.shape)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
(1797, 64)
[0 1 2 ... 8 9 8]
(1797,)


In [12]:
# digits번호가 7이면 True이고 이를 astype(int)로 1로 변환, 7이 아니면 Fasle이고 0으로 변환
y = (
    digits.target == 7 # ndarray 에서 조건식을 넣으면 T or F가 반환
    ).astype(int) # T or F를 int 형으로 바꾸기 때문에 T는 1, F는 0이 됨

X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)
# 7만 맞추면 되는 데이터로 만듦

In [13]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 크기: ', y_test.shape)
print('테스트 세트 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

레이블 크기:  (450,)
테스트 세트 0과 1의 분포도
0    405
1     45
dtype: int64


In [14]:
# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는 :{:.3f}'.format(accuracy_score(y_test, fakepred)))

모든 예측을 0으로 하여도 정확도는 :0.900


# Confusion matirx(오차 행렬)

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]], dtype=int64)

## 정밀도와 재현율

정밀도 = TP / (FP + TP)  
재현율 = TP / (FN + TP)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix


In [17]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {:.4f}, 정밀도: {:.4f}, 재현율: {:.4f}'.format(accuracy, precision, recall))